In [1]:
import pandas as pd
from src.utils.helpers import convert_csv
from src.utils.constants import *
from src.analysis.country_side_map import *

In [2]:
movies = pd.read_csv(DATA_FOLDER_PREPROCESSED + "preprocessed_movies.csv")
convert_csv(movies).head()

,title,languages,countries,genres,release_date,cold_war_side,character_western_bloc_representation,character_eastern_bloc_representation,western_bloc_values,eastern_bloc_values,theme
0,$,NaN,[Russia],"[Crime, Drama, Comedy]",1971,Western,"[Joe Collins, American bank security consultan...","[Dawn Divine, hooker with a heart of gold, cun...",[None],"[Resourcefulness, cleverness, individualism, h...",[None]
1,"$1,000 on the Black","[Italian, German]","[Italy, Germany]",[Western],1966,Eastern,[None],"[Sartana, villainous, oppressive, cruel, arche...","[Johnny Liston, justice, determination, resili...","[Justice, revenge, oppressed vs. oppressor, re...","[Terror, betrayal, familial conflict, crime, r..."
2,"$10,000 Blood Money",NaN,[Russia],"[Drama, Western]",1967,NaN,[None],[None],[None],[None],"[crime, betrayal, revenge, bounty hunter, heis..."
3,"$100,000 for Ringo",[Italian],[Italy],"[Drama, Western]",1965,NaN,[None],[None],[None],[None],"[Western, Civil War, mistaken identity, treasu..."
4,'Anna' i wampir,NaN,[Russia],[Crime],1982,NaN,[None],[None],[None],[None],"[murder mystery, horror, fog, Poland, 1960s]"


In [3]:
country_counts_df = compute_side_movie_count_per_country(movies)

# Display result
print(country_counts_df.head())

                    Country  Occurrences  Western  Eastern  None
0                    Russia        10853      720     2324  7809
1                     Italy         1374      139       71  1164
2                   Germany          645       68       42   535
3  United States of America         8239     1819      140  6280
4                   Estonia           72        7       13    52


In [4]:
display_country_side_map(country_counts_df)

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…